In [34]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import time
import fastparquet
from scipy.ndimage import median_filter
import matplotlib.pyplot as plt
from PIL import Image
from imgaug import augmenters as iaa
import imgaug as ia
import gc
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization

In [22]:
import keras
keras.__version__

'2.2.4'

In [2]:
training_labels = pd.read_csv('./train.csv')
training_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200840 entries, 0 to 200839
Data columns (total 5 columns):
image_id               200840 non-null object
grapheme_root          200840 non-null int64
vowel_diacritic        200840 non-null int64
consonant_diacritic    200840 non-null int64
grapheme               200840 non-null object
dtypes: int64(3), object(2)
memory usage: 7.7+ MB


In [3]:
enc_grapheme_root = OneHotEncoder()
enc_grapheme_root.fit(training_labels.grapheme_root.values.reshape(-1,1))
labels_grapheme = enc_grapheme_root.transform(training_labels.grapheme_root.values.reshape(-1,1))
labels_grapheme = pd.DataFrame(labels_grapheme.toarray(), 
                               columns=['grapheme_' + str(i) 
                                        for i in range(labels_grapheme.shape[1])])
labels_grapheme.shape

c:\users\y2ee2\anaconda3\envs\devenv\lib\site-packages\sklearn\preprocessing\_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(200840, 168)

In [4]:
enc_vowel_diacritic = OneHotEncoder()
enc_vowel_diacritic.fit(training_labels.vowel_diacritic.values.reshape(-1,1))
labels_vowel = enc_vowel_diacritic.transform(training_labels.vowel_diacritic.values.reshape(-1,1))
labels_vowel = pd.DataFrame(labels_vowel.toarray(), 
                            columns = ['vowel_' + str(i) 
                                       for i in range(labels_vowel.shape[1])])
labels_vowel.shape

c:\users\y2ee2\anaconda3\envs\devenv\lib\site-packages\sklearn\preprocessing\_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(200840, 11)

In [5]:
enc_consonant_diacritic = OneHotEncoder()
enc_consonant_diacritic.fit(training_labels.consonant_diacritic.values.reshape(-1,1))
labels_consonent = enc_consonant_diacritic.transform(training_labels.consonant_diacritic.values.reshape(-1,1))
labels_consonent = pd.DataFrame(labels_consonent.toarray(), 
                                columns = ['consonant_'+str(i) 
                                           for i in range(labels_consonent.shape[1])])
labels_consonent.shape

c:\users\y2ee2\anaconda3\envs\devenv\lib\site-packages\sklearn\preprocessing\_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(200840, 7)

In [6]:
df_train = training_labels[['image_id']]
# df_train['image_id'] = df_train['image_id']+'.jpg'
df_train = pd.concat([df_train, labels_grapheme, labels_vowel, labels_consonent], 
                     axis = 1)
df_train.shape

(200840, 187)

In [7]:
from keras import backend as K
import tensorflow as tf

# Compatible with tensorflow backend

def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true[:,0:168], 1), y_pred[:,0:168], 
                        tf.ones_like(y_pred[:,0:168]))
        pt_0 = tf.where(tf.equal(y_true[:,0:168], 0), y_pred[:,0:168], 
                        tf.zeros_like(y_pred[:,0:168]))
        fl = -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
        
        pt_1 = tf.where(tf.equal(y_true[:,168:179], 1), y_pred[:,168:179], 
                        tf.ones_like(y_pred[:,168:179]))
        pt_0 = tf.where(tf.equal(y_true[:,168:179], 0), y_pred[:,168:179], 
                        tf.zeros_like(y_pred[:,168:179]))
        fl_1 = -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
        
        pt_1 = tf.where(tf.equal(y_true[:,179:186], 1), y_pred[:,179:186], 
                        tf.ones_like(y_pred[:,179:186]))
        pt_0 = tf.where(tf.equal(y_true[:,179:186], 0), y_pred[:,179:186], 
                        tf.zeros_like(y_pred[:,179:186]))
        fl_2 = -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
        
        return 2*fl + fl_1 + fl_2
    return focal_loss_fixed

Using TensorFlow backend.


In [8]:
df_train.head()

,image_id,grapheme_0,grapheme_1,grapheme_2,grapheme_3,grapheme_4,grapheme_5,grapheme_6,grapheme_7,grapheme_8,...,vowel_8,vowel_9,vowel_10,consonant_0,consonant_1,consonant_2,consonant_3,consonant_4,consonant_5,consonant_6
0,Train_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Train_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Train_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Train_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,Train_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
np.random.seed(42)
ia.seed(42)

In [10]:
from sklearn.model_selection import train_test_split

train_df, test_df, lab1, lab2 = train_test_split(df_train, 
                                                 df_train.image_id, 
                                                 test_size=0.2, random_state=42)
train_df.shape, test_df.shape

((160672, 187), (40168, 187))

In [12]:
df_train['image_id'] = df_train['image_id']+'.jpg'

In [16]:
len(df_train.columns[1:])

186

In [30]:
train_datagen=ImageDataGenerator(rescale=1./255.,
                                 zoom_range=0.3, 
                                 shear_range=15, 
                                 rotation_range=25, 
                                 height_shift_range=0.2, 
                                 width_shift_range=0.2, 
                                 validation_split=0.2
                                )

train_generator = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory='./train_images/',
        x_col="image_id",
        y_col=df_train.columns[1:],
        target_size=(128, 128),
        batch_size=32,
        class_mode="other")

Found 200840 images.


In [40]:
from keras.applications import ResNet50
from keras.layers import Dense, Dropout, Input, Flatten, GlobalAveragePooling2D
from keras.models import Model
size = 128

inputs = Input(shape=(size, size, 3))
bn = BatchNormalization()(inputs)
pre_trained_model = ResNet50(weights=None, 
                             include_top=False, 
                             # input_shape=(size, size, 3), 
                             # pooling='avg'
                            )
x = pre_trained_model(inputs)
flat = Flatten()(x)
# flat = pre_trained_model.outputs
dense = Dense(512, activation='relu')(flat)
dense = Dropout(0.2)(dense)
dense = Dense(256, activation='relu')(dense)
dense = Dropout(0.4)(dense)
dense = Dense(186, activation='sigmoid')(dense)

cnn = Model(inputs = inputs, outputs=dense)

cnn.compile(optimizer='adam', 
            loss=[focal_loss(alpha=.25, gamma=2)], 
            metrics=['accuracy'])

cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
flatten_4 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               16777728  
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
__________

In [41]:
gc.collect()

0

In [42]:
training_history = cnn.fit_generator(train_generator, 
                                     steps_per_epoch=3, 
                                     epochs=3,
                                     # validation_data=image_generator_test(batch_size = 16),
                                     # validation_steps=1
                                    )

Epoch 1/3
3/3 [==============================] - 40s 13s/step - loss: inf - acc: 0.0000e+00
Epoch 2/3
3/3 [==============================] - 1s 273ms/step - loss: inf - acc: 0.0000e+00
Epoch 3/3
3/3 [==============================] - 1s 275ms/step - loss: inf - acc: 0.0000e+00
